<a href="https://colab.research.google.com/github/zizilnam/Kaggle_Competition_Ventilator_Pressure_Prediction/blob/main/%5BTPU%5D_bi_lstm_gru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# lstm GRU
https://www.kaggle.com/hrshuvo/dnn-lstm-tpu


In [ ]:
%cd /content/drive/MyDrive/Ventilator Pressure Prediction/bi_lstm_gru_kibae

/content/drive/MyDrive/Ventilator Pressure Prediction/bi_lstm_gru_kibae


In [ ]:
### 참고 EDA 
## 01 To learn or not to learn? : https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/278964
print("01 To learn or not to learn?: ", "https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/278964")
print()

## 01-1) PyTorch LSTM with TensorFlow-like initialization
print("01-1) PyTorch LSTM with TensorFlow-like initialization", "https://www.kaggle.com/junkoda/pytorch-lstm-with-tensorflow-like-initialization")
print()

## 01-2) [01-1의 오리진?] Ventilator Pressure | EDA | LSTM 0.189 🔥
print("Ventilator Pressure | EDA | LSTM 0.189 🔥", "https://www.kaggle.com/dmitryuarov/ventilator-pressure-eda-lstm-0-189/notebook")
print()


## 02-1) finetune of Tensorflow Bidirectional LSTM
print("02-1) finetune of Tensorflow Bidirectional LSTM", 'https://www.kaggle.com/tenffe/finetune-of-tensorflow-bidirectional-lstm')
print()

## 02-2 Ensemble Folds with MEDIAN - [0.153]
print("02-2) Ensemble Folds with MEDIAN - [0.153]", "https://www.kaggle.com/cdeotte/ensemble-folds-with-median-0-153")
print()


## 03 finetune of Tensorflow Bidirectional LSTM
print("03. finetune of Tensorflow Bidirectional LSTM", 'https://www.kaggle.com/tenffe/finetune-of-tensorflow-bidirectional-lstm')
print()

## 용담님 - Single Bi-LSTM Model Pressure Predict(GPU) [Infer] : https://www.kaggle.com/emphymachine/single-bi-lstm-model-pressure-predict-gpu-infer
print("용담님 - Single Bi-LSTM Model Pressure Predict(GPU) [Infer]: ", "https://www.kaggle.com/emphymachine/single-bi-lstm-model-pressure-predict-gpu-infer")
print()

##  Google Ventilator LOFO Feature Importance
print("Google Ventilator LOFO Feature Importance", "https://www.kaggle.com/aerdem4/google-ventilator-lofo-feature-importance")
print()

01 To learn or not to learn?:  https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/278964

01-1) PyTorch LSTM with TensorFlow-like initialization https://www.kaggle.com/junkoda/pytorch-lstm-with-tensorflow-like-initialization

Ventilator Pressure | EDA | LSTM 0.189 🔥 https://www.kaggle.com/dmitryuarov/ventilator-pressure-eda-lstm-0-189/notebook

02-1) finetune of Tensorflow Bidirectional LSTM https://www.kaggle.com/tenffe/finetune-of-tensorflow-bidirectional-lstm

02-2) Ensemble Folds with MEDIAN - [0.153] https://www.kaggle.com/cdeotte/ensemble-folds-with-median-0-153

03. finetune of Tensorflow Bidirectional LSTM https://www.kaggle.com/tenffe/finetune-of-tensorflow-bidirectional-lstm

용담님 - Single Bi-LSTM Model Pressure Predict(GPU) [Infer]:  https://www.kaggle.com/emphymachine/single-bi-lstm-model-pressure-predict-gpu-infer

Google Ventilator LOFO Feature Importance https://www.kaggle.com/aerdem4/google-ventilator-lofo-feature-importance



## Preperation

In [ ]:
# Google Drive Mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Import 해야지?

In [ ]:
## gpu 정보
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
!pip install einops

In [ ]:
# Google Ventilaotr 내에서 마운트 및 로드 

base_path = '/content/drive/MyDrive/Ventilator Pressure Prediction/'

import numpy as np
import pandas as pd
import gc
import os


from einops.layers.keras import Rearrange

from sklearn.model_selection import KFold
from sklearn.preprocessing import RobustScaler, normalize
from sklearn.metrics import mean_absolute_error

import tensorflow as tf
print(tf.__version__)

from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Bidirectional, LayerNormalization, LSTM, Multiply, Dense, Dropout, Input, Concatenate, Add, GRU, BatchNormalization, Conv1D, Embedding

from scipy.signal import butter, filtfilt
from pickle import dump

from IPython.display import display
from warnings import filterwarnings
filterwarnings('ignore')

num_workers = os.cpu_count()
num_workers

2.6.0


40

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
debug = True
n = 100 * 1024 if debug else None

ss = pd.read_csv(base_path + 'sample_submission.csv', nrows= n)
train_org = pd.read_csv(base_path + 'train.csv', nrows= n)
test_org = pd.read_csv(base_path + 'test.csv', nrows= n)

# ss = pd.read_csv(base_path + 'sample_submission.csv')
# train_org = pd.read_csv(base_path + 'train.csv')
# test_org = pd.read_csv(base_path + 'test.csv')

In [ ]:
# train.shape, test.shape, ss.shape

## Save Memory

In [ ]:
# 차오르는 RAM을 아껴볼까?
# save_ram : 말 그대로 RAM을 아끼기 위해, 데이터 다이어트를 할 수 있는 코드이다. 
# 출처1:https://www.kaggle.com/dmitryuarov/tps-soft-voting-xgb-cb-lgbm#Basic-information
# 출처2: https://www.kaggle.com/rinnqd/reduce-memory-usage
# 지금 당장 쓰지 않을 것이다. 
# ㄴ사유: 데이터 타입으로 표현되는 숫자가 다르기 때문에, 스케일링 직후(표현되는 스케일이 비슷하기 때문에), 모델에 넣기 전에 하는 것이 적절하다. 

def save_ram(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2

    print(f"최적화 이후 메모리 사용량: {round(end_mem, 2)}MB")
    print(f"메모리 감소비율: {round(100*(start_mem - end_mem)/(start_mem), 2)}%")

    return df

## add_features

In [ ]:
def add_features(df):
    df['cross']= df['u_in'] * df['u_out']
    df['cross2']= df['time_step'] * df['u_out']
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()
    df['time_step_cumsum'] = df.groupby(['breath_id'])['time_step'].cumsum()
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    print("Step-1...Completed")
    
    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    df['u_out_lag1'] = df.groupby('breath_id')['u_out'].shift(1)
    df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
    df['u_out_lag_back1'] = df.groupby('breath_id')['u_out'].shift(-1)
    df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
    df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
    df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
    df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
    df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
    df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
    df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
    df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
    df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
    df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
    df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
    df = df.fillna(0)
    print("Step-2...Completed")

    df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    df['breath_id__u_in__mean'] = df.groupby(['breath_id'])['u_in'].transform('mean')
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    print("Step-3...Completed")
    
    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    df['u_out_diff1'] = df['u_out'] - df['u_out_lag1']
    df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    df['u_out_diff2'] = df['u_out'] - df['u_out_lag2']
    df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
    df['u_out_diff3'] = df['u_out'] - df['u_out_lag3']
    df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
    df['u_out_diff4'] = df['u_out'] - df['u_out_lag4']
    print("Step-4...Completed")
    
    df['one'] = 1
    df['count'] = (df['one']).groupby(df['breath_id']).cumsum()
    df['u_in_cummean'] =df['u_in_cumsum'] /df['count']

    df['breath_id_lag']=df['breath_id'].shift(1).fillna(0)
    df['breath_id_lag2']=df['breath_id'].shift(2).fillna(0)
    df['breath_id_lagsame']=np.select([df['breath_id_lag']==df['breath_id']],[1],0)
    df['breath_id_lag2same']=np.select([df['breath_id_lag2']==df['breath_id']],[1],0)
    df['breath_id__u_in_lag'] = df['u_in'].shift(1).fillna(0)
    df['breath_id__u_in_lag'] = df['breath_id__u_in_lag'] * df['breath_id_lagsame']
    df['breath_id__u_in_lag2'] = df['u_in'].shift(2).fillna(0)
    df['breath_id__u_in_lag2'] = df['breath_id__u_in_lag2'] * df['breath_id_lag2same']
    print("Step-5...Completed")
    
    df['time_step_diff'] = df.groupby('breath_id')['time_step'].diff().fillna(0)

    g = df.groupby('breath_id')['u_in'].apply(pd.Series)
    
    df['ewm_u_in_mean'] = g.ewm(halflife=9).mean()\
                           .reset_index(level=0, drop=True)
                           

    print("====add_more_features_Begins===")
    df['ewm_u_in_std'] = g.ewm(halflife=10).std()\
                          .reset_index(level=0, drop=True)
    df['ewm_u_in_corr'] = g.ewm(halflife=10).corr()\
                           .reset_index(level=0, drop=True)
    df['rolling_10_mean'] = g.rolling(window=10, min_periods=1).mean()\
                             .reset_index(level=0, drop=True)
    df['rolling_10_max'] = g.rolling(window=10, min_periods=1).max()\
                            .reset_index(level=0, drop=True)
    df['rolling_10_std'] = g.rolling(window=10, min_periods=1).std()\
                            .reset_index(level=0, drop=True)

    df['expand_mean'] = g.expanding(2).mean()\
                         .reset_index(level=0, drop=True)
    df['expand_max'] = g.expanding(2).max()\
                        .reset_index(level=0, drop=True)
    df['expand_std'] = g.expanding(2).std()\
                        .reset_index(level=0, drop=True)
    df['R_new'] = df["R"].apply(lambda val: 1 / val**4)
    
    print("====add_more_features_Completed===")
    
    df[["15_in_sum","15_in_min","15_in_max","15_in_mean"]] = (df\
                                                              .groupby('breath_id')['u_in']\
                                                              .rolling(window=15,min_periods=1)\
                                                              .agg({"15_in_sum":"sum",
                                                                    "15_in_min":"min",
                                                                    "15_in_max":"max",
                                                                    "15_in_mean":"mean"})\
                                                               .reset_index(level=0,drop=True))
    print("Step-6...Completed")
    
    df['u_in_lagback_diff1'] = df['u_in'] - df['u_in_lag_back1']
    df['u_out_lagback_diff1'] = df['u_out'] - df['u_out_lag_back1']
    df['u_in_lagback_diff2'] = df['u_in'] - df['u_in_lag_back2']
    df['u_out_lagback_diff2'] = df['u_out'] - df['u_out_lag_back2']
    print("Step-7...Completed")
    #df['R_new']= df['R'] ##푸아죄유의 법칙 
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
    df = pd.get_dummies(df)
    print("Step-8...Completed")
    
    return df

In [ ]:
train = add_features(train_org)
test  = add_features(test_org)

print(train.shape, test.shape)
del train_org, test_org

Step-1...Completed
Step-2...Completed
Step-3...Completed
Step-4...Completed
Step-5...Completed
====add_more_features_Begins===
====add_more_features_Completed===
Step-6...Completed
Step-7...Completed
Step-8...Completed
Step-1...Completed
Step-2...Completed
Step-3...Completed
Step-4...Completed
Step-5...Completed
====add_more_features_Begins===
====add_more_features_Completed===
Step-6...Completed
Step-7...Completed
Step-8...Completed
(102400, 82) (102400, 81)


In [ ]:
train = save_ram(train, verbose=True)

최적화 이후 메모리 사용량: 14.26MB
메모리 감소비율: 73.5%


In [ ]:
train.head()

,id,breath_id,time_step,u_in,u_out,pressure,cross,cross2,area,time_step_cumsum,u_in_cumsum,u_in_lag1,u_out_lag1,u_in_lag_back1,u_out_lag_back1,u_in_lag2,u_out_lag2,u_in_lag_back2,u_out_lag_back2,u_in_lag3,u_out_lag3,u_in_lag_back3,u_out_lag_back3,u_in_lag4,u_out_lag4,u_in_lag_back4,u_out_lag_back4,breath_id__u_in__max,breath_id__u_in__mean,breath_id__u_in__diffmax,breath_id__u_in__diffmean,u_in_diff1,u_out_diff1,u_in_diff2,u_out_diff2,u_in_diff3,u_out_diff3,u_in_diff4,u_out_diff4,one,...,breath_id_lag,breath_id_lag2,breath_id_lagsame,breath_id_lag2same,breath_id__u_in_lag,breath_id__u_in_lag2,time_step_diff,ewm_u_in_mean,ewm_u_in_std,ewm_u_in_corr,rolling_10_mean,rolling_10_max,rolling_10_std,expand_mean,expand_max,expand_std,R_new,15_in_sum,15_in_min,15_in_max,15_in_mean,u_in_lagback_diff1,u_out_lagback_diff1,u_in_lagback_diff2,u_out_lagback_diff2,R_20,R_5,R_50,C_10,C_20,C_50,R__C_20__10,R__C_20__20,R__C_20__50,R__C_50__10,R__C_50__20,R__C_50__50,R__C_5__10,R__C_5__20,R__C_5__50
0,1,1,0.000000,0.083313,0,5.835938,0.0,0.0,0.000000,0.000000,0.083313,0.000000,0.0,18.390625,0.0,0.000000,0.0,22.515625,0.0,0.000000,0.0,22.812500,0.0,0.000000,0.0,25.359375,0.0,28.3125,10.148438,28.234375,10.062500,0.083313,0.0,0.083313,0.0,0.083313,0.0,0.083313,0.0,1,...,0.0,0.0,0,0,0.000000,0.000000,0.000000,0.083313,NaN,NaN,0.083313,0.083313,NaN,NaN,NaN,NaN,0.000006,0.083313,0.083313,0.083313,0.083313,-18.296875,0.0,-22.421875,0.0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0
1,2,1,0.033661,18.390625,0,5.906250,0.0,0.0,0.618652,0.033661,18.468750,0.083313,0.0,22.515625,0.0,0.000000,0.0,22.812500,0.0,0.000000,0.0,25.359375,0.0,0.000000,0.0,27.265625,0.0,28.3125,10.148438,9.929688,-8.234375,18.296875,0.0,18.390625,0.0,18.390625,0.0,18.390625,0.0,1,...,1.0,0.0,1,0,0.083313,0.000000,0.033661,9.585938,12.937500,1.0,9.234375,18.390625,12.937500,9.234375,18.390625,12.937500,0.000006,18.468750,0.083313,18.390625,9.234375,-4.125000,0.0,-4.425781,0.0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0
2,3,1,0.067505,22.515625,0,7.875000,0.0,0.0,2.138672,0.101196,40.968750,18.390625,0.0,22.812500,0.0,0.083313,0.0,25.359375,0.0,0.000000,0.0,27.265625,0.0,0.000000,0.0,27.125000,0.0,28.3125,10.148438,5.804688,-12.359375,4.125000,0.0,22.421875,0.0,22.515625,0.0,22.515625,0.0,1,...,1.0,1.0,1,1,18.390625,0.083313,0.033875,14.226562,11.781250,1.0,13.656250,22.515625,11.937500,13.656250,22.515625,11.937500,0.000006,40.968750,0.083313,22.515625,13.656250,-0.299561,0.0,-2.845703,0.0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0
3,4,1,0.101562,22.812500,0,11.742188,0.0,0.0,4.453125,0.202759,63.781250,22.515625,0.0,25.359375,0.0,18.390625,0.0,27.265625,0.0,0.083313,0.0,27.125000,0.0,0.000000,0.0,26.812500,0.0,28.3125,10.148438,5.503906,-12.664062,0.299561,0.0,4.425781,0.0,22.718750,0.0,22.812500,0.0,1,...,1.0,1.0,1,1,22.515625,18.390625,0.034027,16.625000,10.445312,1.0,15.945312,22.812500,10.765625,15.945312,22.812500,10.765625,0.000006,63.781250,0.083313,22.812500,15.945312,-2.546875,0.0,-4.449219,0.0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0
4,5,1,0.135742,25.359375,0,12.234375,0.0,0.0,7.898438,0.338379,89.125000,22.812500,0.0,27.265625,0.0,22.515625,0.0,27.125000,0.0,18.390625,0.0,26.812500,0.0,0.083313,0.0,27.859375,0.0,28.3125,10.148438,2.957031,-15.210938,2.546875,0.0,2.845703,0.0,6.972656,0.0,25.265625,0.0,1,...,1.0,1.0,1,1,22.812500,22.515625,0.034210,18.656250,9.804688,1.0,17.828125,25.359375,10.226562,17.828125,25.359375,10.226562,0.000006,89.125000,0.083313,25.359375,17.828125,-1.904297,0.0,-1.771484,0.0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0


In [ ]:
test = save_ram(test)

최적화 이후 메모리 사용량: 14.06MB
메모리 감소비율: 73.48%


In [ ]:
test.head()

,id,breath_id,time_step,u_in,u_out,cross,cross2,area,time_step_cumsum,u_in_cumsum,u_in_lag1,u_out_lag1,u_in_lag_back1,u_out_lag_back1,u_in_lag2,u_out_lag2,u_in_lag_back2,u_out_lag_back2,u_in_lag3,u_out_lag3,u_in_lag_back3,u_out_lag_back3,u_in_lag4,u_out_lag4,u_in_lag_back4,u_out_lag_back4,breath_id__u_in__max,breath_id__u_in__mean,breath_id__u_in__diffmax,breath_id__u_in__diffmean,u_in_diff1,u_out_diff1,u_in_diff2,u_out_diff2,u_in_diff3,u_out_diff3,u_in_diff4,u_out_diff4,one,count,...,breath_id_lag,breath_id_lag2,breath_id_lagsame,breath_id_lag2same,breath_id__u_in_lag,breath_id__u_in_lag2,time_step_diff,ewm_u_in_mean,ewm_u_in_std,ewm_u_in_corr,rolling_10_mean,rolling_10_max,rolling_10_std,expand_mean,expand_max,expand_std,R_new,15_in_sum,15_in_min,15_in_max,15_in_mean,u_in_lagback_diff1,u_out_lagback_diff1,u_in_lagback_diff2,u_out_lagback_diff2,R_20,R_5,R_50,C_10,C_20,C_50,R__C_20__10,R__C_20__20,R__C_20__50,R__C_50__10,R__C_50__20,R__C_50__50,R__C_5__10,R__C_5__20,R__C_5__50
0,1,0,0.000000,0.000000,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,7.515625,0.0,0.000000,0.0,14.648438,0.0,0.000000,0.0,21.234375,0.0,0.0,0.0,26.328125,0.0,37.53125,9.328125,37.531250,9.328125,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,1,1,...,0.0,0.0,1,1,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,0.0016,0.000000,0.0,0.000000,0.000000,-7.515625,0.0,-14.648438,0.0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
1,2,0,0.031891,7.515625,0,0.0,0.0,0.239746,0.031891,7.515625,0.000000,0.0,14.648438,0.0,0.000000,0.0,21.234375,0.0,0.000000,0.0,26.328125,0.0,0.0,0.0,30.484375,0.0,37.53125,9.328125,30.031250,1.812500,7.515625,0.0,7.515625,0.0,7.515625,0.0,7.515625,0.0,1,2,...,0.0,0.0,1,1,0.000000,0.000000,0.031891,3.902344,5.312500,1.0,3.757812,7.515625,5.312500,3.757812,7.515625,5.312500,0.0016,7.515625,0.0,7.515625,3.757812,-7.136719,0.0,-13.718750,0.0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
2,3,0,0.063843,14.648438,0,0.0,0.0,1.174805,0.095703,22.171875,7.515625,0.0,21.234375,0.0,0.000000,0.0,26.328125,0.0,0.000000,0.0,30.484375,0.0,0.0,0.0,33.531250,0.0,37.53125,9.328125,22.890625,-5.324219,7.136719,0.0,14.648438,0.0,14.648438,0.0,14.648438,0.0,1,3,...,0.0,0.0,1,1,7.515625,0.000000,0.031921,7.765625,7.320312,1.0,7.390625,14.648438,7.328125,7.390625,14.648438,7.328125,0.0016,22.171875,0.0,14.648438,7.390625,-6.578125,0.0,-11.671875,0.0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
3,4,0,0.095764,21.234375,0,0.0,0.0,3.207031,0.191528,43.406250,14.648438,0.0,26.328125,0.0,7.515625,0.0,30.484375,0.0,0.000000,0.0,33.531250,0.0,0.0,0.0,35.718750,0.0,37.53125,9.328125,16.312500,-11.906250,6.578125,0.0,13.718750,0.0,21.234375,0.0,21.234375,0.0,1,4,...,0.0,0.0,1,1,14.648438,7.515625,0.031921,11.531250,9.125000,1.0,10.851562,21.234375,9.148438,10.851562,21.234375,9.148438,0.0016,43.406250,0.0,21.234375,10.851562,-5.089844,0.0,-9.257812,0.0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
4,5,0,0.127686,26.328125,0,0.0,0.0,6.566406,0.319092,69.687500,21.234375,0.0,30.484375,0.0,14.648438,0.0,33.531250,0.0,7.515625,0.0,35.718750,0.0,0.0,0.0,36.968750,0.0,37.53125,9.328125,11.218750,-17.000000,5.089844,0.0,11.671875,0.0,18.812500,0.0,26.328125,0.0,1,5,...,0.0,0.0,1,1,21.234375,14.648438,0.031891,14.960938,10.437500,1.0,13.945312,26.328125,10.515625,13.945312,26.328125,10.515625,0.0016,69.687500,0.0,26.328125,13.945312,-4.164062,0.0,-7.226562,0.0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0


In [ ]:
test.head(2)

,id,breath_id,time_step,u_in,u_out,cross,cross2,area,time_step_cumsum,u_in_cumsum,u_in_lag1,u_out_lag1,u_in_lag_back1,u_out_lag_back1,u_in_lag2,u_out_lag2,u_in_lag_back2,u_out_lag_back2,u_in_lag3,u_out_lag3,u_in_lag_back3,u_out_lag_back3,u_in_lag4,u_out_lag4,u_in_lag_back4,u_out_lag_back4,breath_id__u_in__max,breath_id__u_in__mean,breath_id__u_in__diffmax,breath_id__u_in__diffmean,u_in_diff1,u_out_diff1,u_in_diff2,u_out_diff2,u_in_diff3,u_out_diff3,u_in_diff4,u_out_diff4,one,count,...,breath_id_lag,breath_id_lag2,breath_id_lagsame,breath_id_lag2same,breath_id__u_in_lag,breath_id__u_in_lag2,time_step_diff,ewm_u_in_mean,ewm_u_in_std,ewm_u_in_corr,rolling_10_mean,rolling_10_max,rolling_10_std,expand_mean,expand_max,expand_std,R_new,15_in_sum,15_in_min,15_in_max,15_in_mean,u_in_lagback_diff1,u_out_lagback_diff1,u_in_lagback_diff2,u_out_lagback_diff2,R_20,R_5,R_50,C_10,C_20,C_50,R__C_20__10,R__C_20__20,R__C_20__50,R__C_50__10,R__C_50__20,R__C_50__50,R__C_5__10,R__C_5__20,R__C_5__50
0,1,0,0.000000,0.000000,0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,7.515625,0.0,0.0,0.0,14.648438,0.0,0.0,0.0,21.234375,0.0,0.0,0.0,26.328125,0.0,37.53125,9.328125,37.53125,9.328125,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,1,1,...,0.0,0.0,1,1,0.0,0.0,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,0.0016,0.000000,0.0,0.000000,0.000000,-7.515625,0.0,-14.648438,0.0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
1,2,0,0.031891,7.515625,0,0.0,0.0,0.239746,0.031891,7.515625,0.0,0.0,14.648438,0.0,0.0,0.0,21.234375,0.0,0.0,0.0,26.328125,0.0,0.0,0.0,30.484375,0.0,37.53125,9.328125,30.03125,1.812500,7.515625,0.0,7.515625,0.0,7.515625,0.0,7.515625,0.0,1,2,...,0.0,0.0,1,1,0.0,0.0,0.031891,3.902344,5.3125,1.0,3.757812,7.515625,5.3125,3.757812,7.515625,5.3125,0.0016,7.515625,0.0,7.515625,3.757812,-7.136719,0.0,-13.718750,0.0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0


## **[국룰] add_features**

In [ ]:
# ## https://www.kaggle.com/tenffe/finetune-of-tensorflow-bidirectional-lstm/notebook

# def add_features(df):
#     df['area'] = df['time_step'] * df['u_in']
#     df['area'] = df.groupby('breath_id')['area'].cumsum()
    
#     df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    
#     df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
#     df['u_out_lag1'] = df.groupby('breath_id')['u_out'].shift(1)
#     df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
#     df['u_out_lag_back1'] = df.groupby('breath_id')['u_out'].shift(-1)
#     df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
#     df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
#     df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
#     df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
#     df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
#     df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
#     df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
#     df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
#     df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
#     df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
#     df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
#     df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
#     df = df.fillna(0)
    
#     df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
#     df['breath_id__u_out__max'] = df.groupby(['breath_id'])['u_out'].transform('max')
    
#     df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
#     df['u_out_diff1'] = df['u_out'] - df['u_out_lag1']
#     df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
#     df['u_out_diff2'] = df['u_out'] - df['u_out_lag2']
    
#     df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
#     df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    
#     df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
#     df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    
#     df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
#     df['u_out_diff3'] = df['u_out'] - df['u_out_lag3']
#     df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
#     df['u_out_diff4'] = df['u_out'] - df['u_out_lag4']
#     df['cross']= df['u_in']*df['u_out']
#     df['cross2']= df['time_step']*df['u_out']
    
#     df['R'] = df['R'].astype(str)
#     df['C'] = df['C'].astype(str)
#     df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
#     df = pd.get_dummies(df)

# # https://www.kaggle.com/junkoda/pytorch-lstm-with-tensorflow-like-initialization/notebook
#     # g = df.groupby('breath_id')['u_in'].apply(pd.Series)
#     # df['ewm_u_in_mean'] = g.ewm(halflife=10).mean()\
#     #                        .reset_index(level=0, drop=True)
#     # df['ewm_u_in_std'] = g.ewm(halflife=10).std()\
#     #                       .reset_index(level=0, drop=True)
#     # df['ewm_u_in_corr'] = g.ewm(halflife=10).corr()\
#     #                        .reset_index(level=0, drop=True)
#     # df['rolling_10_mean'] = g.rolling(window=10, min_periods=1).mean()\
#     #                          .reset_index(level=0, drop=True)
#     # df['rolling_10_max'] = g.rolling(window=10, min_periods=1).max()\
#     #                         .reset_index(level=0, drop=True)
#     # df['rolling_10_std'] = g.rolling(window=10, min_periods=1).std()\
#     #                         .reset_index(level=0, drop=True)

#     # df['expand_mean'] = g.expanding(2).mean()\
#     #                      .reset_index(level=0, drop=True)
#     # df['expand_max'] = g.expanding(2).max()\
#     #                     .reset_index(level=0, drop=True)
#     # df['expand_std'] = g.expanding(2).std()\
#     #                     .reset_index(level=0, drop=True)
#     # df = df.fillna(0)



#     df.drop(['id', 'breath_id'], axis=1, inplace=True)
#     if 'pressure' in df.columns:
#         df.drop('pressure', axis=1, inplace=True)

        
#     return df



In [ ]:
# train = add_features(train)
# test = add_features(test)

## **[국룰 + 갓용담님버전] add_features**

In [ ]:
# ## https://www.kaggle.com/emphymachine/single-bi-lstm-model-pressure-predict-gpu-infer

# def add_features(df):
#     df['area'] = df['time_step'] * df['u_in']
#     df['area'] = df.groupby('breath_id')['area'].cumsum()
#     df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
#     df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
#     df['u_out_lag1'] = df.groupby('breath_id')['u_out'].shift(1)
#     df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
#     df['u_out_lag_back1'] = df.groupby('breath_id')['u_out'].shift(-1)
#     df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
#     df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
#     df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
#     df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
#     df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
#     df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
#     df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
#     df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
#     df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
#     df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
#     df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
#     df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
#     df = df.fillna(0)
#     df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
#     df['breath_id__u_out__max'] = df.groupby(['breath_id'])['u_out'].transform('max')
#     df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
#     df['u_out_diff1'] = df['u_out'] - df['u_out_lag1']
#     df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
#     df['u_out_diff2'] = df['u_out'] - df['u_out_lag2']
#     df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
#     df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
#     df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
#     df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
#     df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
#     df['u_out_diff3'] = df['u_out'] - df['u_out_lag3']
#     df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
#     df['u_out_diff4'] = df['u_out'] - df['u_out_lag4']
#     df['cross']= df['u_in']*df['u_out']
#     df['cross2']= df['time_step']*df['u_out']
#     df['R'] = df['R'].astype(str)
#     df['C'] = df['C'].astype(str)
#     df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
#     df = pd.get_dummies(df)


#     df.drop(['id', 'breath_id'], axis=1, inplace=True)
#     if 'pressure' in df.columns:
#         df.drop('pressure', axis=1, inplace=True)


#     return df

### 필요없는 feature drop


In [ ]:
## target
targets = train[['pressure']].to_numpy().reshape(-1, 80)
targets.shape

(1280, 80)

In [ ]:
## train
train.drop(['pressure', 'id', 'breath_id','one','count','breath_id_lag','breath_id_lag2','breath_id_lagsame',
            'breath_id_lag2same'], axis=1, inplace=True)

In [ ]:
## 확인
print(train.shape)
train.head(1)

(102400, 73)


,time_step,u_in,u_out,cross,cross2,area,time_step_cumsum,u_in_cumsum,u_in_lag1,u_out_lag1,u_in_lag_back1,u_out_lag_back1,u_in_lag2,u_out_lag2,u_in_lag_back2,u_out_lag_back2,u_in_lag3,u_out_lag3,u_in_lag_back3,u_out_lag_back3,u_in_lag4,u_out_lag4,u_in_lag_back4,u_out_lag_back4,breath_id__u_in__max,breath_id__u_in__mean,breath_id__u_in__diffmax,breath_id__u_in__diffmean,u_in_diff1,u_out_diff1,u_in_diff2,u_out_diff2,u_in_diff3,u_out_diff3,u_in_diff4,u_out_diff4,u_in_cummean,breath_id__u_in_lag,breath_id__u_in_lag2,time_step_diff,ewm_u_in_mean,ewm_u_in_std,ewm_u_in_corr,rolling_10_mean,rolling_10_max,rolling_10_std,expand_mean,expand_max,expand_std,R_new,15_in_sum,15_in_min,15_in_max,15_in_mean,u_in_lagback_diff1,u_out_lagback_diff1,u_in_lagback_diff2,u_out_lagback_diff2,R_20,R_5,R_50,C_10,C_20,C_50,R__C_20__10,R__C_20__20,R__C_20__50,R__C_50__10,R__C_50__20,R__C_50__50,R__C_5__10,R__C_5__20,R__C_5__50
0,0.0,0.083313,0,0.0,0.0,0.0,0.0,0.083313,0.0,0.0,18.390625,0.0,0.0,0.0,22.515625,0.0,0.0,0.0,22.8125,0.0,0.0,0.0,25.359375,0.0,28.3125,10.148438,28.234375,10.0625,0.083313,0.0,0.083313,0.0,0.083313,0.0,0.083313,0.0,0.083313,0.0,0.0,0.0,0.083313,NaN,NaN,0.083313,0.083313,NaN,NaN,NaN,NaN,0.000006,0.083313,0.083313,0.083313,0.083313,-18.296875,0.0,-22.421875,0.0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0


In [ ]:
## Test
test = test.drop(['id', 'breath_id','one','count','breath_id_lag','breath_id_lag2','breath_id_lagsame',
                  'breath_id_lag2same'], axis=1)

In [ ]:
## 확인
print(test.shape)
test.head(1)

(102400, 73)


,time_step,u_in,u_out,cross,cross2,area,time_step_cumsum,u_in_cumsum,u_in_lag1,u_out_lag1,u_in_lag_back1,u_out_lag_back1,u_in_lag2,u_out_lag2,u_in_lag_back2,u_out_lag_back2,u_in_lag3,u_out_lag3,u_in_lag_back3,u_out_lag_back3,u_in_lag4,u_out_lag4,u_in_lag_back4,u_out_lag_back4,breath_id__u_in__max,breath_id__u_in__mean,breath_id__u_in__diffmax,breath_id__u_in__diffmean,u_in_diff1,u_out_diff1,u_in_diff2,u_out_diff2,u_in_diff3,u_out_diff3,u_in_diff4,u_out_diff4,u_in_cummean,breath_id__u_in_lag,breath_id__u_in_lag2,time_step_diff,ewm_u_in_mean,ewm_u_in_std,ewm_u_in_corr,rolling_10_mean,rolling_10_max,rolling_10_std,expand_mean,expand_max,expand_std,R_new,15_in_sum,15_in_min,15_in_max,15_in_mean,u_in_lagback_diff1,u_out_lagback_diff1,u_in_lagback_diff2,u_out_lagback_diff2,R_20,R_5,R_50,C_10,C_20,C_50,R__C_20__10,R__C_20__20,R__C_20__50,R__C_50__10,R__C_50__20,R__C_50__50,R__C_5__10,R__C_5__20,R__C_5__50
0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.515625,0.0,0.0,0.0,14.648438,0.0,0.0,0.0,21.234375,0.0,0.0,0.0,26.328125,0.0,37.53125,9.328125,37.53125,9.328125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0016,0.0,0.0,0.0,0.0,-7.515625,0.0,-14.648438,0.0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0


## scaler

In [ ]:
sc = RobustScaler()
train = sc.fit_transform(train)
test = sc.transform(test) ## np.ndarray

In [ ]:
train = train.reshape(-1, 80, train.shape[-1])
train.shape

(1280, 80, 73)

In [ ]:
test = test.reshape(-1, 80, train.shape[-1])
test.shape

(1280, 80, 73)

In [ ]:
targets.shape

(1280, 80)

In [ ]:
# test.shape[-2:]

In [ ]:
pressure = targets.squeeze().reshape(-1, 1).astype('float32')

P_MIN = np.min(pressure)
P_MAX = np.max(pressure)
P_STEP = (pressure[1] -  pressure[0])[0]

print(f"MIN Pressure: {P_MIN}")
print(f"MAX Pressure: {P_MAX}")
print(f"Pressure step: {P_STEP}")
print(f"Unique Values: {np.unique(pressure).shape[0]}")

MIN Pressure: -1.8955078125
MAX Pressure: 64.8125
Pressure step: 0.0703125
Unique Values: 806


In [ ]:
del pressure

In [ ]:
gc.collect()

202

## TPU or NOT?

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
    BATCH_SIZE = tpu_strategy.num_replicas_in_sync * 64
    print("Running on TPU:", tpu.master())
    print(f"Batch Size: {BATCH_SIZE}")
    
except ValueError:
    strategy = tf.distribute.get_strategy()
    BATCH_SIZE = 512
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    print(f"Batch Size: {BATCH_SIZE}")

INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.103.90.162:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.103.90.162:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU: grpc://10.103.90.162:8470
Batch Size: 512


## Model

In [ ]:

def rc_to_index(r,c):
    ## 가중치?
    #r_map = { 5: 0, 20: 1, 50: 2}
    #c_map = {10: 0, 20: 1, 50: 2}
    r = (r== 5).float()*1 + (r==20).float()*2 + (r==50).float()*3
    c = (c==10).float()*1 + (c==20).float()*2 + (c==50).float()*3
    r = r.long()
    c = c.long()
    return r,c

In [ ]:
# train.shape
a = Input(shape=(train.shape[-2:]))
a

<KerasTensor: shape=(None, 80, 73) dtype=float32 (created by layer 'input_1')>

In [ ]:
train.shape

(1280, 80, 73)

In [ ]:
def d_model():
    
    x_input = Input(shape=(train.shape[-2:]))
    ## 입력 형태: 80 * train.shape[-1]

    # r_embed = Embedding(4, 2) #, padding_idx=0)
    # c_embed = Embedding(4, 2) #, padding_idx=0)
    #seq_embed = Sequential()
    #seq_embed.add(Rearrange('b l d -> b d l'))
    #seq_embed.add(Conv1D( 32, kernel_size = 5, strides=1, padding='same', activation ='selu'))
    #seq_embed.add(Rearrange('b d l -> b l d'))
    #seq_embed.add(LayerNormalization(32))
    #seq_embed.add(Activation('tanh'))
        # nn.SiLU(),
        # nn.Dropout(0.),
    #)

    # r,c = rc_to_index(x_input[:,:,'R'],x_input[:,:,'C'])
    # r = r_embed(r)
    # c = c_embed(c)
    # seq = torch.cat((r, c, x[:,:,:-2]), 2)
    #seq = Concatenate(axis=2)(x_input[:,:,:-2])
    #x_input2 = seq_embed(seq)


    x1 = Bidirectional(LSTM(units = 768, return_sequences = True))(x_input)
    x2 = Bidirectional(LSTM(units = 512, return_sequences = True))(x1)

    ## z2랑
    x3 = Bidirectional(LSTM(units = 384, return_sequences = True))(x2)

    ## z3랑
    x4 = Bidirectional(LSTM(units = 256, return_sequences = True))(x3)

    ## z4랑 
    x5 = Bidirectional(LSTM(units=128, return_sequences=True))(x4)
    
    z2 = Bidirectional(GRU(units=384, return_sequences=True))(x2)
    
    z31 = Multiply()([x3, z2])
    z31 = BatchNormalization()(z31)

    z3 = Bidirectional(GRU(units=256, return_sequences=True))(z31)
    
    z41 = Multiply()([x4, z3])
    z41 = BatchNormalization()(z41)
    
    z4 = Bidirectional(GRU(units=128, return_sequences=True))(z41)
    
    z51 = Multiply()([x5, z4])
    z51 = BatchNormalization()(z51)
    
    z5 = Bidirectional(GRU(units=64, return_sequences=True))(z51)
    
    x = Concatenate(axis=2)([x5, z2, z3, z4, z5])
    
    x = Dense(units=128, activation='selu')(x)
    
    x_output = Dense(units=1)(x)


    model = Model(inputs=x_input, outputs=x_output, name='BLG_Model')

    return model

In [ ]:
model = d_model()
model.summary()

Model: "BLG_Model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 80, 73)]     0                                            
__________________________________________________________________________________________________
bidirectional_63 (Bidirectional (None, 80, 1536)     5173248     input_9[0][0]                    
__________________________________________________________________________________________________
bidirectional_64 (Bidirectional (None, 80, 1024)     8392704     bidirectional_63[0][0]           
__________________________________________________________________________________________________
bidirectional_65 (Bidirectional (None, 80, 768)      4328448     bidirectional_64[0][0]           
__________________________________________________________________________________________

## Train & Predict

In [ ]:
train.shape

(1280, 80, 73)

In [ ]:
with tpu_strategy.scope():
    
    VERBOSE = 0
    test_preds = []
    kf = KFold(n_splits=7, shuffle=True, random_state=4401)
    
    for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
        X_train, X_valid = train[train_idx], train[test_idx]
        y_train, y_valid = targets[train_idx], targets[test_idx]
        
        model = d_model()
        model.compile(optimizer="adam", loss="mae")

        lr = ReduceLROnPlateau(monitor="val_loss", factor=0.85, 
                               patience=7, verbose=VERBOSE)
        
        save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
        chk_point = ModelCheckpoint(f'./Bidirect_LSTM_model_{fold+1}C.h5', options=save_locally, 
                                    monitor='val_loss', verbose=VERBOSE, 
                                    save_best_only=True, mode='min')

        es = EarlyStopping(monitor="val_loss", patience=30, 
                           verbose=VERBOSE, mode="min", 
                           restore_best_weights=True)
        
        model.fit(X_train, y_train, 
                  validation_data=(X_valid, y_valid), 
                  epochs=400,
                  verbose=VERBOSE,
                  batch_size=BATCH_SIZE, 
                  callbacks=[lr, chk_point, es])
        
        load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
        model = load_model(f'./Bidirect_LSTM_model_{fold+1}C.h5', options=load_locally)
        
        y_true = y_valid.squeeze().reshape(-1, 1)
        y_pred = model.predict(X_valid, batch_size=BATCH_SIZE).squeeze().reshape(-1, 1)
        score = mean_absolute_error(y_true, y_pred)
        print(f"Fold-{fold+1} - Score: {score}")
        
        test_preds.append(model.predict(test, batch_size=BATCH_SIZE).squeeze().reshape(-1, 1).squeeze())

        if i >= 1:
            break

OSError: ignored

## Submit

In [ ]:
base_path

In [ ]:
sample = pd.read_csv(base_path + 'sample_submission.csv')

In [ ]:
sample["pressure"] = np.median(np.vstack(test_preds),axis=0)
sample["pressure"] = np.round((sample.pressure - P_MIN)/P_STEP) * P_STEP + P_MIN
sample["pressure"] = np.clip(sample.pressure, P_MIN, P_MAX)
sample.to_csv(base_path + 'blg_tpu_median_submission.csv', index=False)
sample.to_csv(base_path + 'blg_tpu_submission.csv', index=False)